# Logistic regression using StatsModels

In this Notebook we are going to perform an epidemiological study assessing the possible clinical association between coronary heart disease after 10 years and sBP.
Previous data shows high sBP may be risk factor for coronary heart disease. 

This experiment is an unique opportunity to leverage data from the Framingham cohort (the first and one of the largest epidemiological cohorts - https://en.wikipedia.org/wiki/Framingham_Heart_Study) and run some models in Python using the StatsModels package.

### First let's import all important packages

In [1]:
import numpy as np 
import pandas as pd
import statsmodels.formula.api as smf

# Exploring data

### Open the dataset

In [2]:
df = pd.read_csv('framingham.csv')

### Show data frame

In [3]:
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


The head function shows us all columns and the first 5 rows/observations of our data frame.

In [4]:
df.shape

(4240, 16)

The output means the data frame has 4240 observations (rows) and 16 variables (columns). Remember **.head()** only gave us a glimpse from the 4240 rows.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4240 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             4240 non-null   int64  
 1   age              4240 non-null   int64  
 2   education        4135 non-null   float64
 3   currentSmoker    4240 non-null   int64  
 4   cigsPerDay       4211 non-null   float64
 5   BPMeds           4187 non-null   float64
 6   prevalentStroke  4240 non-null   int64  
 7   prevalentHyp     4240 non-null   int64  
 8   diabetes         4240 non-null   int64  
 9   totChol          4190 non-null   float64
 10  sysBP            4240 non-null   float64
 11  diaBP            4240 non-null   float64
 12  BMI              4221 non-null   float64
 13  heartRate        4239 non-null   float64
 14  glucose          3852 non-null   float64
 15  TenYearCHD       4240 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 530.1 KB


In [6]:
df.describe()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,4240.000000,4240.000000,4135.000000,4240.000000,4211.000000,4187.000000,4240.000000,4240.000000,4240.000000,4190.000000,4240.000000,4240.000000,4221.000000,4239.000000,3852.000000,4240.000000
mean,0.429245,49.580189,1.979444,0.494104,9.005937,0.029615,0.005896,0.310613,0.025708,236.699523,132.354599,82.897759,25.800801,75.878981,81.963655,0.151887
std,0.495027,8.572942,1.019791,0.500024,11.922462,0.169544,0.076569,0.462799,0.158280,44.591284,22.033300,11.910394,4.079840,12.025348,23.954335,0.358953
min,0.000000,32.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,107.000000,83.500000,48.000000,15.540000,44.000000,40.000000,0.000000
25%,0.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.000000,75.000000,23.070000,68.000000,71.000000,0.000000
50%,0.000000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.000000,82.000000,25.400000,75.000000,78.000000,0.000000
75%,1.000000,56.000000,3.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,263.000000,144.000000,90.000000,28.040000,83.000000,87.000000,0.000000
max,1.000000,70.000000,4.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000


**Categorical (non-binary) variable: Education

**Discrete binary variables; male, CurrentSmoker, BPMeds, prevalentStroke, prevalentHyp, diabetes and TenYearCHD. 

The empty parenthesis in **df.describe()** tells Python to show all available variables.

Here we can see the number of observations per column (count), followed by the mean and standard deviation (sd). Note that for binary discrete variables the interpretation is somewhat different and the mean will equal the relative proportion (0 to 1, instead of 0–100%). 

### Simple Logistic Regression 

In [8]:
model1 = smf.logit('TenYearCHD ~ sysBP', df).fit()

Optimization terminated successfully.
         Current function value: 0.404872
         Iterations 6


In [9]:
print(model1.summary()) 

                           Logit Regression Results                           
Dep. Variable:             TenYearCHD   No. Observations:                 4240
Model:                          Logit   Df Residuals:                     4238
Method:                           MLE   Df Model:                            1
Date:                Sun, 17 Apr 2022   Pseudo R-squ.:                 0.04953
Time:                        14:39:38   Log-Likelihood:                -1716.7
converged:                       True   LL-Null:                       -1806.1
Covariance Type:            nonrobust   LLR p-value:                 8.447e-41
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.0001      0.256    -19.542      0.000      -5.502      -4.499
sysBP          0.0241      0.002     13.358      0.000       0.021       0.028


In [10]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 4240
Model:                          Logit   Df Residuals:                     4238
Method:                           MLE   Df Model:                            1
Date:                Sun, 17 Apr 2022   Pseudo R-squ.:                 0.04953
Time:                        14:40:03   Log-Likelihood:                -1716.7
converged:                       True   LL-Null:                       -1806.1
Covariance Type:            nonrobust   LLR p-value:                 8.447e-41
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.0001      0.256    -19.542      0.000      -5.502      -4.499
sysBP          0.0241      0.002     13.358      0.000       0.021       0.028
==============================================================================
"""

In [12]:
params = np.exp(model1.params)
conf = np.exp(model1.conf_int())
conf['OR'] = params
pvalue=round(model1.pvalues,3)
conf['pvalue']=pvalue
conf.columns = ['CI 95%(2.5%)', 'CI 95%(97.5%)', 'Odds Ratio','pvalue']
print ((conf))

           CI 95%(2.5%)  CI 95%(97.5%)  Odds Ratio  pvalue
Intercept      0.004081       0.011125    0.006738     0.0
sysBP          1.020740       1.027971    1.024349     0.0


In [13]:
conf = np.exp(model1.conf_int())
conf

,0,1
Intercept,0.004081,0.011125
sysBP,1.020740,1.027971


### Multiple logistic regression

In [14]:
model2 = smf.logit(formula='TenYearCHD ~ sysBP + age + male + C(education) + cigsPerDay + totChol', data=df).fit()

# any problem to fix?

Optimization terminated successfully.
         Current function value: 0.380164
         Iterations 7


In [15]:
print(model2.summary())

                           Logit Regression Results                           
Dep. Variable:             TenYearCHD   No. Observations:                 4059
Model:                          Logit   Df Residuals:                     4050
Method:                           MLE   Df Model:                            8
Date:                Sun, 17 Apr 2022   Pseudo R-squ.:                  0.1080
Time:                        14:47:20   Log-Likelihood:                -1543.1
converged:                       True   LL-Null:                       -1729.8
Covariance Type:            nonrobust   LLR p-value:                 8.567e-76
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -8.4051      0.449    -18.709      0.000      -9.286      -7.525
C(education)[T.2.0]    -0.1980      0.116     -1.705      0.088      -0.426       0.030
C(education)[T.3

With the help of Numpy (np), we are calling Odds ratios with their 95% confidence interval (OR(95%CI))

With a line of code we can transform all coefficients into OR

In [17]:
np.exp(model2.params)

Intercept              0.000224
C(education)[T.2.0]    0.820361
C(education)[T.3.0]    0.911478
C(education)[T.4.0]    0.995764
sysBP                  1.017807
age                    1.069217
male                   1.600458
cigsPerDay             1.021016
totChol                1.001952
dtype: float64

However, we would prefer to have more information besides OR, namely the 95%CI with its lower and upper bounds and the p-value

In [19]:
params = np.exp(model2.params)
conf = np.exp(model2.conf_int())
conf['OR'] = params
pvalue=round(model2.pvalues,3)
conf['pvalue']=pvalue
conf.columns = ['CI 95%(2.5%)', 'CI 95%(97.5%)', 'Odds Ratio','pvalue']
print ((conf))

                     CI 95%(2.5%)  CI 95%(97.5%)  Odds Ratio  pvalue
Intercept                0.000093       0.000540    0.000224   0.000
C(education)[T.2.0]      0.653382       1.030013    0.820361   0.088
C(education)[T.3.0]      0.694637       1.196010    0.911478   0.504
C(education)[T.4.0]      0.736157       1.346922    0.995764   0.978
sysBP                    1.013749       1.021880    1.017807   0.000
age                      1.056241       1.082352    1.069217   0.000
male                     1.311191       1.953540    1.600458   0.000
cigsPerDay               1.013153       1.028941    1.021016   0.000
totChol                  0.999854       1.004054    1.001952   0.068


In [17]:
model3 = smf.glm(formula='TenYearCHD ~ sysBP + C(education) + cigsPerDay + totChol + male*age', data=df).fit()

In [18]:
print(model3.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             TenYearCHD   No. Observations:                 4059
Model:                            GLM   Df Residuals:                     4049
Model Family:                Gaussian   Df Model:                            9
Link Function:               identity   Scale:                         0.11730
Method:                          IRLS   Log-Likelihood:                -1405.3
Date:                Sun, 17 Apr 2022   Deviance:                       474.97
Time:                        14:55:39   Pearson chi2:                     475.
No. Iterations:                     3   Pseudo R-squ. (CS):            0.09636
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -0.5602    

In [19]:
params = np.exp(model3.params)
conf = np.exp(model3.conf_int())
conf['OR'] = params
pvalue=round(model3.pvalues,3)
conf['pvalue']=pvalue
conf.columns = ['CI 95%(2.5%)', 'CI 95%(97.5%)', 'Odds Ratio','pvalue']
print ((conf))

                     CI 95%(2.5%)  CI 95%(97.5%)  Odds Ratio  pvalue
Intercept                0.517099       0.630722    0.571092   0.000
C(education)[T.2.0]      0.954293       1.005124    0.979379   0.116
C(education)[T.3.0]      0.958100       1.019383    0.988267   0.456
C(education)[T.4.0]      0.964240       1.035432    0.999202   0.965
sysBP                    1.002074       1.003137    1.002606   0.000
cigsPerDay               1.001609       1.003526    1.002567   0.000
totChol                  0.999884       1.000394    1.000139   0.285
male                     0.774764       1.002164    0.881158   0.054
age                      1.004181       1.007966    1.006072   0.000
male:age                 1.000960       1.006109    1.003531   0.007


**easier steps and elegant tables?**

Presentation-Ready Data Summary and Analytic Result Tables • gtsummary
http://www.danieldsjoberg.com/gtsummary/ 

**can we use it in python?**

Leveraging the best of both Python and R - DataCamp
https://www.datacamp.com/community/tutorials/using-both-python-r  